In [ ]:
import itertools
from pathlib import Path

import pandas as pd

from portfolio_factor_allocation.backtesting import block_bootstrap_metrics

In [ ]:
data_dir = Path.cwd().parent / "data"
returns_dir = data_dir / "returns"
bootstrap_dir = data_dir / "bootstrap"

(bootstrap_dir / "sharpe").mkdir(parents=True, exist_ok=True)
(bootstrap_dir / "volatility").mkdir(parents=True, exist_ok=True)
(bootstrap_dir / "information_ratio").mkdir(parents=True, exist_ok=True)
(bootstrap_dir / "tracking_error").mkdir(parents=True, exist_ok=True)

### Load data

In [ ]:
df_market = pd.read_csv(data_dir / "returns" / "market.csv", parse_dates=["date"])
df_market["date"] = df_market["date"].dt.to_period("M")
df_monthly_rf = pd.read_csv(data_dir / "monthly_rf.csv", parse_dates=["date"])
df_monthly_rf["date"] = df_monthly_rf["date"].dt.to_period("M")

### Block bootstrapping

In [ ]:
factors = ["V", "W", "C", "R", "L"]
factor_combs = [
    list(c)
    for r in range(2, len(factors) + 1)
    for c in itertools.combinations(factors, r)
]

In [3]:
factor_combs = [["V", "W"]]
suffixes = [
    "_int_ter",
    "_mix_ter",
    "_int_dec",
    "_mix_dec",
    "_int_bw",
    "_mix_bw",
]
block_size = 5  # In months
n_sim = 1000
r_f = 0.0
seed = 42

In [ ]:
market_series = df_market.set_index("date")["market"]
rf_series = df_monthly_rf.set_index("date")["rf"]

for factor_comb in factor_combs:
    factor_str = "_".join(factor_comb)
    returns_cols = [factor_str + s for s in suffixes]
    df_r = pd.read_csv(returns_dir / f"{factor_str}.csv")
    df_r["date"] = pd.to_datetime(df_r["date"]).dt.to_period("M")
    df_r = df_r.set_index("date")

    # Align with market and rf
    common_index = df_r.index.intersection(market_series.index).intersection(
        rf_series.index
    )

    aligned_r = df_r.loc[common_index]
    aligned_m = market_series.loc[common_index]
    aligned_rf = rf_series.loc[common_index]

    # Perform block bootstrapping
    df_r_subset = aligned_r[returns_cols]
    results = block_bootstrap_metrics(
        df_r_subset, aligned_m, block_size, n_sim, r_f=aligned_rf, seed=seed
    )

    # Save results
    results["sharpe"].to_csv(
        bootstrap_dir / "sharpe" / f"{factor_str}.csv", index=False
    )
    results["volatility"].to_csv(
        bootstrap_dir / "volatility" / f"{factor_str}.csv", index=False
    )
    results["information_ratio"].to_csv(
        bootstrap_dir / "information_ratio" / f"{factor_str}.csv", index=False
    )
    results["tracking_error"].to_csv(
        bootstrap_dir / "tracking_error" / f"{factor_str}.csv", index=False
    )